In [1]:
from os import fsencode, listdir, fsdecode

import cv2
import imutils
from PIL import Image

import numpy as np

directory = fsencode('input')
for file in listdir(directory):
    filename = fsdecode(file)
    print(filename)
    if filename == 'IMG_1926.jpg':
        file_path = 'input/' + filename
        image = cv2.imread(file_path)
        # pil_img = Image.open(file_path)
        # samples = 0
        # for i in range(1000, 1010):
        #     print(pil_img.getpixel((i, 10)))
        # break
        ratio = image.shape[0] / 300.0
        image = imutils.resize(image, height=300)
        #blurred = cv2.pyrMeanShiftFiltering(image, 31, 91)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        #blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        # sigma = 0.88
        # v = np.median(blurred)
        # lower = int(max(0, (1.0 - sigma) * v))
        # upper = int(min(255, (1.0 + sigma) * v))
        # lower_mask = [160, 180, 200]
        # upper_mask = [170, 185, 210]
        # white_mask = cv2.inRange(gray, lower_mask, upper_mask)
        #thresh = cv2.threshold(gray, 100, 200, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blurred = cv2.bilateralFilter(gray, 11, 17, 17)
        # blur = cv2.GaussianBlur(image, (1, 1), 1000)
        # flag, thresh = cv2.threshold(blur, 33, 100, cv2.THRESH_BINARY)
        #cv2.imshow('Grayed', thresh)
        #cv2.waitKey(100)
        edged = cv2.Canny(blurred, 30, 200)
        cv2.imshow('Edged', edged)
        cv2.waitKey(100)
        # lines = cv2.HoughLinesP(edged,1,np.pi/4,15,100,50)
        # for x in range(0, len(lines)):
        #     for x1,y1,x2,y2 in lines[x]:
        #         cv2.line(image,(x1,y1),(x2,y2),(0,255,0),2)
        # cv2.imshow('hough',image)
        # cv2.waitKey(100)
        (returned_edged, contours, hierarchy) = cv2.findContours(edged.copy(), cv2.RETR_LIST,
                                                                 cv2.CHAIN_APPROX_NONE)
        largest_contours = sorted(contours, key=cv2.contourArea, reverse=True)
        # print (largest_contours)
        image_contour = None
        for c in largest_contours:
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)
            if len(approx) == 4:
                image_contour = approx
                break
        print(image_contour)
        cv2.drawContours(image, [image_contour], -1, (0, 255, 0), 1)
        cv2.imshow('Contour', image)
        cv2.waitKey(100)
        crop_x_min = min(image_contour[0][0][0], image_contour[3][0][0]) * ratio
        crop_x_max = max(image_contour[1][0][0], image_contour[2][0][0]) * ratio
        crop_y_min = min(image_contour[0][0][1], image_contour[1][0][1]) * ratio
        crop_y_max = max(image_contour[2][0][1], image_contour[3][0][1]) * ratio
        try:
            original = Image.open(file_path)
            cropped = original.crop(
                (crop_x_min, crop_y_min, min(crop_x_max, original.size[0]), min(crop_y_max, original.size[1])))
            cropped.save('output/' + filename)
        except Exception:
            pass


IMG_3125.jpg
IMG_1926.jpg


None


error: OpenCV(3.4.1) /io/opencv/modules/imgproc/src/drawing.cpp:2612: error: (-215) reader.ptr != __null in function cvDrawContours
